In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
from pathlib import Path
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
import joblib

# ───────────────────────────────────────────────
# ROOT du projet (FORCÉ)
# ───────────────────────────────────────────────
ROOT = Path(r"C:\Users\Balerion\Desktop\us-flights-delay").resolve()
assert (ROOT / "src").exists(), f"❌ Dossier src introuvable sous {ROOT}"

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

print("[INFO] Project Root =", ROOT)
print("[INFO] src exists? ", (ROOT / "src").exists())

from src.config import (
    DATA_PROCESSED_DIR,
    MODELS_DIR,
    MLFLOW_EXPERIMENT_NAME,
    MLFLOW_TRACKING_URI_CFG,
)

from src.mlflow_utils import setup_mlflow

print("[INFO] DATA_PROCESSED_DIR :", DATA_PROCESSED_DIR)
print("[INFO] MODELS_DIR :", MODELS_DIR)
print("[INFO] MLFLOW URI (cfg) :", MLFLOW_TRACKING_URI_CFG)
print("[INFO] EXPERIMENT NAME :", MLFLOW_EXPERIMENT_NAME)

# Sécurité
if mlflow.active_run():
    mlflow.end_run()

try:
    mlflow.autolog(disable=True)
except:
    pass

setup_mlflow()

print("[MLflow] Tracking URI effectif :", mlflow.get_tracking_uri())
print("[MLflow] Experiment :", MLFLOW_EXPERIMENT_NAME)

[INFO] Project Root = C:\Users\Balerion\Desktop\us-flights-delay
[INFO] src exists?  True
[INFO] ROOT = C:\Users\Balerion\Desktop\us-flights-delay
[INFO] src exists?  True
[INFO] config.yml chargé depuis : C:\Users\Balerion\Desktop\us-flights-delay\config.yml
[INFO] DATA_RAW_DIR      = C:\Users\Balerion\Desktop\us-flights-delay\data\raw
[INFO] DATA_PROCESSED_DIR= C:\Users\Balerion\Desktop\us-flights-delay\data\processed
[INFO] WEATHER_DIR       = C:\Users\Balerion\Desktop\us-flights-delay\data\weather
[INFO] MODELS_DIR        = C:\Users\Balerion\Desktop\us-flights-delay\models
[INFO] REPORTS_DIR       = C:\Users\Balerion\Desktop\us-flights-delay\reports
[INFO] MLRUNS_DIR        = C:\Users\Balerion\Desktop\us-flights-delay\mlruns
[INFO] MLFLOW_TRACKING_URI (cfg) = file:///C:/Users/Balerion/Desktop/us-flights-delay/mlruns
[INFO] MLflow experiment name    = us_flights_delay
[INFO] HIGH_DELAY_RATE_THRESHOLD = 0.25
[INFO] WEATHER_CACHE_DIR         = C:\Users\Balerion\Desktop\us-flights-dela

In [2]:
train_w_path = DATA_PROCESSED_DIR / "train_weather.csv"
val_w_path   = DATA_PROCESSED_DIR / "val_weather.csv"
test_w_path  = DATA_PROCESSED_DIR / "test_weather.csv"

for p in [train_w_path, val_w_path, test_w_path]:
    if not p.exists():
        raise FileNotFoundError(f"❌ Fichier manquant : {p}")

train = pd.read_csv(train_w_path)
val   = pd.read_csv(val_w_path)
test  = pd.read_csv(test_w_path)

print("\n[INFO] SHAPES (avec météo) :")
print("train :", train.shape)
print("val   :", val.shape)
print("test  :", test.shape)

train.head()


[INFO] SHAPES (avec météo) :
train : (119998, 34)
val   : (25714, 34)
test  : (25714, 34)


,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,divert_rate,carrier_delay_per_flight,weather_delay_per_flight,nas_delay_per_flight,security_delay_per_flight,late_aircraft_delay_per_flight,high_delay_risk,temperature_2m_mean,precipitation_sum,windspeed_10m_max
0,2015,10,AS,Alaska Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",132.0,20.0,4.66,0.00,...,0.000000,3.068182,0.000000,2.704545,0.000,0.742424,0,21.651613,6.380645,17.874194
1,2018,9,OH,PSA Airlines Inc.,CHA,"Chattanooga, TN: Lovell Field",136.0,26.0,6.73,2.28,...,0.000000,2.830882,0.911765,0.588235,0.000,9.750000,0,25.600000,5.556667,12.570000
2,2016,1,NK,Spirit Air Lines,MCI,"Kansas City, MO: Kansas City International",93.0,32.0,9.59,0.10,...,0.000000,6.096774,0.107527,4.763441,0.000,6.860215,1,-1.774194,0.622581,18.048387
3,2018,8,UA,United Air Lines Inc.,IAH,"Houston, TX: George Bush Intercontinental/Houston",5391.0,1197.0,285.63,65.08,...,0.001669,3.719533,1.156372,3.348729,0.000,7.092933,1,28.070968,2.525806,16.716129
4,2014,6,MQ,Envoy Air,SPS,"Wichita Falls, TX: Sheppard AFB/Wichita Falls ...",120.0,36.0,12.89,6.31,...,0.000000,3.925000,3.116667,1.625000,0.075,5.008333,1,27.846667,3.683333,27.313333


In [3]:
WEATHER_COLS = [
    "temperature_2m_mean",
    "precipitation_sum",
    "windspeed_10m_max",
]

print("=== Vérification colonnes météo dans les splits ===")
for name, df in [("TRAIN", train), ("VAL", val), ("TEST", test)]:
    missing = [c for c in WEATHER_COLS if c not in df.columns]
    if missing:
        print(f"❌ {name} — colonnes météo manquantes :", missing)
    else:
        print(f"✔ {name} — OK")

def weather_nan_summary(df, name):
    existing = [c for c in WEATHER_COLS if c in df.columns]
    return (
        df[existing]
        .isna()
        .mean()
        .to_frame(name)
        .T
        .assign(split=name)
        .set_index("split")
    )

summary_nan = pd.concat(
    [
        weather_nan_summary(train, "train"),
        weather_nan_summary(val, "val"),
        weather_nan_summary(test, "test"),
    ]
)
summary_nan

=== Vérification colonnes météo dans les splits ===
✔ TRAIN — OK
✔ VAL — OK
✔ TEST — OK


,temperature_2m_mean,precipitation_sum,windspeed_10m_max
split,,,
train,0.0,0.0,0.0
val,0.0,0.0,0.0
test,0.0,0.0,0.0


In [4]:
exp = mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
if exp is None:
    raise ValueError(f"❌ Experiment '{MLFLOW_EXPERIMENT_NAME}' introuvable")

experiment_id = exp.experiment_id
print("[INFO] Experiment ID :", experiment_id)

df_runs = mlflow.search_runs(
    experiment_ids=[experiment_id],
    output_format="pandas",
)

print("[INFO] Nombre total de runs MLflow :", len(df_runs))

# ───────────────────────────────────────────
# CORRECTION : si 'tags.stage' n'existe pas
# ───────────────────────────────────────────
if "tags.stage" in df_runs.columns:
    df_final = df_runs[df_runs["tags.stage"] == "final"].copy()
    print("[INFO] Runs filtrés (stage=final) :", len(df_final))
else:
    print("[WARN] Aucun tag 'stage' trouvé → Tous les runs seront utilisés.")
    df_final = df_runs.copy()

[INFO] Experiment ID : 174010619286583251
[INFO] Nombre total de runs MLflow : 6
[WARN] Aucun tag 'stage' trouvé → Tous les runs seront utilisés.


In [5]:
# Colonnes réellement présentes dans df_final
available_cols = set(df_final.columns)

# Colonnes souhaitées mais facultatives
optional_cols = [
    "tags.task",
    "params.model_type",
]

# Colonnes obligatoires
base_cols = [
    "run_id",
    "tags.mlflow.runName",
    "metrics.test_auc",
    "metrics.test_rmse",
]

# On ne garde que celles qui existent VRAIMENT
cols_keep = [c for c in base_cols + optional_cols if c in df_final.columns]

print("[INFO] Colonnes disponibles :", df_final.columns.tolist())
print("[INFO] Colonnes utilisées dans df_display :", cols_keep)

df_display = df_final[cols_keep].copy()

df_display = df_display.rename(
    columns={
        "tags.mlflow.runName": "run_name",
        "tags.task": "task",
        "params.model_type": "model_type",
        "metrics.test_auc": "test_auc",
        "metrics.test_rmse": "test_rmse",
    }
)

df_display = df_display.sort_values(["run_id"]).reset_index(drop=True)
df_display

[INFO] Colonnes disponibles : ['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time', 'end_time', 'metrics.test_mae', 'metrics.test_r2', 'metrics.test_rmse', 'metrics.test_accuracy', 'metrics.test_auc', 'metrics.test_recall', 'params.reg_lambda', 'params.max_depth', 'params.gamma', 'params.min_child_weight', 'params.colsample_bytree', 'params.reg_alpha', 'params.learning_rate', 'params.subsample', 'params.n_estimators', 'tags.mlflow.source.name', 'tags.mlflow.source.type', 'tags.mlflow.user', 'tags.mlflow.runName']
[INFO] Colonnes utilisées dans df_display : ['run_id', 'tags.mlflow.runName', 'metrics.test_auc', 'metrics.test_rmse']


,run_id,run_name,test_auc,test_rmse
0,069a7e7917d2419c820eb788fc0bbd3d,xgb_classifier_optuna_weather,0.999993,NaN
1,192d60c5fec941b2bc52c23d863e9f45,xgb_regressor_optuna_weather,NaN,0.949102
2,25031b5e8cab48829e99bd3a8f531b85,xgb_classifier_optuna_weather,0.999993,NaN
3,70e104926a17410389204aaa0e95cd39,xgb_classifier_optuna_weather,0.999994,NaN
4,7232223c9d2b40b789b78fc5646e34ed,xgb_regressor_optuna_weather,NaN,0.872593
5,c245f35b5b4d43f6b9c52fc4ee298b74,xgb_classifier_optuna_weather,0.999992,NaN


In [6]:
def get_best_classifier(df):
    df_auc = df.dropna(subset=["test_auc"])
    if df_auc.empty:
        raise ValueError("❌ Aucun run avec test_auc")

    idx = df_auc["test_auc"].idxmax()
    return df_auc.loc[idx]

def get_best_regressor(df):
    df_rmse = df.dropna(subset=["test_rmse"])
    if df_rmse.empty:
        raise ValueError("❌ Aucun run avec test_rmse")

    idx = df_rmse["test_rmse"].idxmin()
    return df_rmse.loc[idx]

best_clf = get_best_classifier(df_display)
best_reg = get_best_regressor(df_display)

print(best_clf)
print(best_reg)

run_id       70e104926a17410389204aaa0e95cd39
run_name        xgb_classifier_optuna_weather
test_auc                             0.999994
test_rmse                                 NaN
Name: 3, dtype: object
run_id       7232223c9d2b40b789b78fc5646e34ed
run_name         xgb_regressor_optuna_weather
test_auc                                  NaN
test_rmse                            0.872593
Name: 4, dtype: object


In [7]:
clf_path = MODELS_DIR / "xgb_classifier_optuna_weather.joblib"
reg_path = MODELS_DIR / "xgb_regressor_optuna_weather.joblib"

print("[INFO] MODELS_DIR      =", MODELS_DIR)
print("[INFO] CLASSIFIER PATH =", clf_path)
print("[INFO] REGRESSOR PATH  =", reg_path)

if not clf_path.exists() or not reg_path.exists():
    raise FileNotFoundError("❌ Modèles absents → relancer train_optuna.py")

clf_model = joblib.load(clf_path)
reg_model = joblib.load(reg_path)

print("[INFO] Modèles chargés :", type(clf_model), type(reg_model))

[INFO] MODELS_DIR      = C:\Users\Balerion\Desktop\us-flights-delay\models
[INFO] CLASSIFIER PATH = C:\Users\Balerion\Desktop\us-flights-delay\models\xgb_classifier_optuna_weather.joblib
[INFO] REGRESSOR PATH  = C:\Users\Balerion\Desktop\us-flights-delay\models\xgb_regressor_optuna_weather.joblib
[INFO] Modèles chargés : <class 'sklearn.pipeline.Pipeline'> <class 'sklearn.pipeline.Pipeline'>


In [8]:
# On récupère exactement les colonnes utilisées à l'entraînement
from src.features import get_feature_columns

FEATURE_COLS = get_feature_columns()
print("[INFO] Colonnes utilisées pour la prédiction :", FEATURE_COLS)

# Vérification stricte
missing = [c for c in FEATURE_COLS if c not in test.columns]
if missing:
    raise ValueError(f"❌ Colonnes manquantes dans test_weather : {missing}")
else:
    print("✔ Toutes les colonnes nécessaires sont présentes.")

# Construction d'un sample conforme
sample = test[FEATURE_COLS].iloc[[0]].copy()

print("\n=== Vérification du SAMPLE ===")
print("Nombre de colonnes dans sample :", len(sample.columns))
print("Nombre attendu (features.py)   :", len(FEATURE_COLS))

if len(sample.columns) != len(FEATURE_COLS):
    raise ValueError("❌ sample n’a pas le même nombre de colonnes que FEATURES_COLS")

print("\n=== Liste exacte des colonnes du sample ===")
for c in sample.columns:
    print("-", c)

print("\n=== Exemple de ligne test_weather (features complètes) ===")
display(sample)


# Fonction prédiction
def predict_delay(sample_df, clf, reg):
    proba = float(clf.predict_proba(sample_df)[0, 1])
    avg_delay = float(reg.predict(sample_df)[0])
    return {
        "high_delay_proba": proba,
        "avg_delay_minutes": avg_delay,
    }

# Prédictions
pred = predict_delay(sample, clf_model, reg_model)

print("\n=== PREDICTIONS (sample[0]) ===")
print(pred)

[INFO] Colonnes utilisées pour la prédiction : ['arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'temperature_2m_mean', 'precipitation_sum', 'windspeed_10m_max', 'delay_rate', 'avg_delay_per_flight', 'carrier', 'carrier_name', 'airport', 'airport_name', 'year', 'month']
✔ Toutes les colonnes nécessaires sont présentes.

=== Vérification du SAMPLE ===
Nombre de colonnes dans sample : 26
Nombre attendu (features.py)   : 26

=== Liste exacte des colonnes du sample ===
- arr_flights
- arr_del15
- carrier_ct
- weather_ct
- nas_ct
- security_ct
- late_aircraft_ct
- arr_cancelled
- arr_diverted
- arr_delay
- carrier_delay
- weather_delay
- nas_delay
- security_delay
- late_aircraft_delay
- temperature_2m_mean
- precipitation_sum
- windspeed_10m_max
- delay_rate
- avg_delay_per_flight
- carrier
- carrier_name
-

,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,...,precipitation_sum,windspeed_10m_max,delay_rate,avg_delay_per_flight,carrier,carrier_name,airport,airport_name,year,month
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9E,Endeavor Air Inc.,DHN,"Dothan, AL: Dothan Regional",2018,4



=== PREDICTIONS (sample[0]) ===
{'high_delay_proba': 5.552766651817365e-06, 'avg_delay_minutes': 0.008667546324431896}


In [9]:
summary_models = pd.DataFrame([
    {
        "model": "XGBClassifier_optuna_weather",
        "task": "Classification (high_delay_risk)",
        "run_id": best_clf["run_id"],
        "metric": "AUC",
        "value": best_clf["test_auc"],
    },
    {
        "model": "XGBRegressor_optuna_weather",
        "task": "Régression (avg_delay_per_flight)",
        "run_id": best_reg["run_id"],
        "metric": "RMSE ↓",
        "value": best_reg["test_rmse"],
    },
])

summary_models

,model,task,run_id,metric,value
0,XGBClassifier_optuna_weather,Classification (high_delay_risk),70e104926a17410389204aaa0e95cd39,AUC,0.999994
1,XGBRegressor_optuna_weather,Régression (avg_delay_per_flight),7232223c9d2b40b789b78fc5646e34ed,RMSE ↓,0.872593


In [10]:
slides_bullets = f"""
### MLflow — Résumé des modèles finaux (Optuna + météo)

- Tracking MLflow : `{mlflow.get_tracking_uri()}`
- Expérience MLflow : `{MLFLOW_EXPERIMENT_NAME}`

#### 🔵 Classification — high_delay_risk
- Modèle : **XGBClassifier_optuna_weather**
- Meilleur run : `{best_clf["run_id"]}`
- AUC (test) : **{best_clf["test_auc"]:.5f}**

#### 🟣 Régression — avg_delay_per_flight
- Modèle : **XGBRegressor_optuna_weather**
- Meilleur run : `{best_reg["run_id"]}`
- RMSE (test) : **{best_reg["test_rmse"]:.3f} min**

#### 🔥 Features météo intégrées
- temperature_2m_mean
- precipitation_sum
- windspeed_10m_max

Pipeline :
1. Préparation & split
2. Intégration météo Open-Meteo
3. Optuna hyperparamètres XGBoost
4. MLflow tracking (runs, metrics, artifacts)
5. Sauvegarde : mlruns + models/joblib
6. Chargement standardisé pour API / Streamlit
"""

print(slides_bullets)


### MLflow — Résumé des modèles finaux (Optuna + météo)

- Tracking MLflow : `file:///C:/Users/Balerion/Desktop/us-flights-delay/mlruns`
- Expérience MLflow : `us_flights_delay`

#### 🔵 Classification — high_delay_risk
- Modèle : **XGBClassifier_optuna_weather**
- Meilleur run : `70e104926a17410389204aaa0e95cd39`
- AUC (test) : **0.99999**

#### 🟣 Régression — avg_delay_per_flight
- Modèle : **XGBRegressor_optuna_weather**
- Meilleur run : `7232223c9d2b40b789b78fc5646e34ed`
- RMSE (test) : **0.873 min**

#### 🔥 Features météo intégrées
- temperature_2m_mean
- precipitation_sum
- windspeed_10m_max

Pipeline :
1. Préparation & split
2. Intégration météo Open-Meteo
3. Optuna hyperparamètres XGBoost
4. MLflow tracking (runs, metrics, artifacts)
5. Sauvegarde : mlruns + models/joblib
6. Chargement standardisé pour API / Streamlit

